In [12]:
import buffett_code_com as bc

import requests
import json
import pandas as pd
from datetime import *

ticker = 6594

In [25]:
# Generate company data
def generate_df_c_all():
    res_c_all = bc.fetch(bc.API_ENDPOINT_COMPANY, ticker,  None, None)
    json_data_c_all = json.loads(res_c_all.text)
    # json に保管
    with open('json/company.json', 'w') as f:
        json.dump(json_data_c_all, f)

In [26]:
# API からの取得 JSON への保存
generate_df_c_all()

In [4]:
# 全期間の有価証券情報を Dynamo に追加する

In [27]:
import decimal

# JSON からの取得
json_c_all = json.load(open('json/company.json', 'r'),  parse_float=decimal.Decimal)
json_c_target = json_c_all[str(ticker)]
df_c = pd.DataFrame.from_dict(json_c_target)
df_c["fixed_tier_range"][0]

{'oldest_fiscal_year': Decimal('2015.0'),
 'oldest_fiscal_quarter': Decimal('4.0'),
 'latest_fiscal_year': Decimal('2020.0'),
 'latest_fiscal_quarter': Decimal('2.0')}

In [28]:
# Generate quarter data
def generate_df_q(ticker:str, start_q:str, end_q:str):
    res_q = bc.fetch(bc.API_ENDPOINT_QUARTER, ticker, start_q, end_q)
    json_data_q = json.loads(res_q.text,  parse_float=decimal.Decimal)
    df_q = pd.DataFrame.from_dict(json_data_q[ticker])
    # 日付は edinet_updated_date から取得
    df_q['to_datetime'] = pd.to_datetime(df_q['edinet_updated_date'])
    df_sorted = sorted(df_q['to_datetime'] )
    df_concat = pd.DataFrame()
    for sort_num in range(len(df_sorted)):
        datetime_data = df_q[df_q['to_datetime'] == df_sorted[sort_num]]
        df_concat = pd.concat([df_concat, datetime_data], axis=0)
    return df_q

In [29]:
#  oldest_fiscal_year / oldest_fiscal_quarter= 2004.0 1.0
# latest_fiscal_year / latest_fiscal_quarter = 2020.0 2.0
df_q = generate_df_q(str(ticker), "2015Q4", "2018Q3")
df_q
# for column in df_q.columns:
#    print(column)

,company_name,ceo_name,headquarters_address,end_date,updated_date,edinet_updated_date,tdnet_updated_date,edinet_title,tdnet_title,accounting_standard,...,employee_num,net_sales_per_employee,operating_income_per_employee,segment_member,increase_in_properties,r_and_d_expenses,desc_business,fiscal_year,fiscal_quarter,to_datetime
0,日本電産株式会社,代表取締役会長兼社長 永守 重信,京都府京都市南区久世殿城町３３８番地,2016-03-31T00:00:00.000Z,2016-06-20T00:00:00.000Z,2016-06-20T00:00:00.000Z,2016-04-25T15:15:00.000Z,【E01975】日本電産株式会社 有価証券報告書－第43期(平成27年4月1日－平成28年3...,【65940】日本電産株式会社 平成28年3月期決算短信〔米国基準〕（連結）,US GAAP,...,96602.0,12197366.514150845,1289186.5592844868,None,81918000000.0,51978000000.0,"\n<h3>３【事業の内容】</h3>\n<p style=""margin-left: 12...",2015.0,4.0,2016-06-20 00:00:00+00:00
1,日本電産株式会社,代表取締役会長兼社長 永守 重信,京都市南区久世殿城町３３８番地,2016-06-30T00:00:00.000Z,2016-08-09T00:00:00.000Z,2016-08-09T00:00:00.000Z,2016-07-22T15:15:00.000Z,【E01975】日本電産株式会社 四半期報告書－第44期第1四半期(平成28年4月1日－平成...,【65940】日本電産株式会社 平成29年3月期 第1四半期決算短信〔ＩＦＲＳ〕（連結）,IFRS,...,None,None,None,None,0.0,12660000000.0,None,2016.0,1.0,2016-08-09 00:00:00+00:00
2,日本電産株式会社,代表取締役会長兼社長 永守 重信,京都市南区久世殿城町３３８番地,2016-09-30T00:00:00.000Z,2016-11-11T00:00:00.000Z,2016-11-11T00:00:00.000Z,2016-10-24T15:15:00.000Z,【E01975】日本電産株式会社 四半期報告書－第44期第2四半期(平成28年7月1日－平成...,【65940】日本電産株式会社 平成29年3月期 第2四半期決算短信〔ＩＦＲＳ〕（連結）,IFRS,...,None,None,None,None,30483000000.0,25602000000.0,None,2016.0,2.0,2016-11-11 00:00:00+00:00
3,日本電産株式会社,代表取締役会長兼社長 永守 重信,京都市南区久世殿城町３３８番地,2016-12-31T00:00:00.000Z,2017-02-13T00:00:00.000Z,2017-02-13T00:00:00.000Z,2017-01-24T15:15:00.000Z,【E01975】日本電産株式会社 四半期報告書－第44期第3四半期(平成28年10月1日－平...,【65940】日本電産株式会社 平成29年3月期 第3四半期決算短信〔ＩＦＲＳ〕（連結）,IFRS,...,None,None,None,None,0.0,38760000000.0,None,2016.0,3.0,2017-02-13 00:00:00+00:00
4,日本電産株式会社,代表取締役会長兼社長 永守 重信,京都市南区久世殿城町３３８番地,2017-03-31T00:00:00.000Z,2017-06-21T16:14:00.000Z,2017-06-21T16:14:00.000Z,2017-04-25T15:15:00.000Z,【E01975】日本電産株式会社 訂正有価証券報告書－第44期(平成28年4月1日－平成29...,【65940】日本電産株式会社 平成29年3月期決算短信〔ＩＦＲＳ〕（連結）,IFRS,...,107062.0,11202023.126786347,1310745.1756925893,None,68718000000.0,52807000000.0,"\n<h3>３【事業の内容】</h3>\n<p style=""margin-left: 12...",2016.0,4.0,2017-06-21 16:14:00+00:00
5,日本電産株式会社,代表取締役会長兼社長 永守 重信,京都市南区久世殿城町３３８番地,2017-06-30T00:00:00.000Z,2017-08-09T15:38:00.000Z,2017-08-09T15:38:00.000Z,2017-07-26T15:15:00.000Z,【E01975】日本電産株式会社 四半期報告書－第45期第1四半期(平成29年4月1日－平成...,【65940】日本電産株式会社 平成30年3月期 第1四半期決算短信〔ＩＦＲＳ〕（連結）,IFRS,...,None,None,None,None,0.0,13244000000.0,None,2017.0,1.0,2017-08-09 15:38:00+00:00
6,日本電産株式会社,代表取締役会長兼社長 永守 重信,京都市南区久世殿城町３３８番地,2017-09-30T00:00:00.000Z,2017-11-13T15:31:00.000Z,2017-11-13T15:31:00.000Z,2017-10-24T15:15:00.000Z,【E01975】日本電産株式会社 四半期報告書－第45期第2四半期(平成29年7月1日－平成...,【65940】日本電産株式会社 平成30年3月期 第2四半期決算短信〔ＩＦＲＳ〕（連結）,IFRS,...,None,None,None,None,41841000000.0,26789000000.0,None,2017.0,2.0,2017-11-13 15:31:00+00:00
7,日本電産株式会社,代表取締役会長兼社長 永守 重信,京都市南区久世殿城町３３８番地,2017-12-31T00:00:00.000Z,2018-02-13T15:40:00.000Z,2018-02-13T15:40:00.000Z,2018-01-24T15:15:00.000Z,【E01975】日本電産株式会社 四半期報告書－第45期第3四半期(平成29年10月1日－平...,【65940】日本電産株式会社 平成30年3月期 第3四半期決算短信〔ＩＦＲＳ〕（連結）,IFRS,...,None,None,None,None,0.0,41006000000.0,"\n<h3>２【事業の内容】</h3>\n<p style=""margin-left: 24...",2017.0,3.0,2018-02-13 15:40:00+00:00
8,日本電産株式会社,代表取締役会長（最高経営責任者） 永守 重信,京都市南区久世殿城町３３８番地,2018-03-31T00:00:00.000Z,2018-06-21T15:46:00.000Z,2018-06-21T15:46:00.000Z,2018-04-24T15:15:00.000Z,【E01975】日本電産株式会社 有価証券報告書－第45期(平成29年4月1日－平成30年3...,【65940】日本電産株式会社 平成30年3月期決算短信〔ＩＦＲＳ〕（連結）,IFRS,...,107554.0,13835747.624449113,1551239.3774290124,None,90841000000.0,55438000000.0,"\n<h3>３【事業の内容】</h3>\n<p style=""margin-left: 12...",2017.0,4.0,2018-06-21 15:46:00+00:00
9,日本電産株式会社,代表取締役会長（最高経営責任者) 永守 重信,京都市南区久世殿城町３３８番地,2018-06-30T00:00:00.000Z,2018-08-09T15:37:00.000Z,2018-08-09T15:37:00.000Z,2018-07-25T15:15:00.000Z,【E01975】日本電産株式会社 四半期報告書－第46期第1四半期(平成30年4月1日－平成...,【65940】日本電産株式会社 2019年3月期 第1四半期決算短信〔IFRS〕(連結),IFRS,...,None,None,None,None,32875000000.0,15166000000.0,None,2018.0,1.0,2018-08-09 15:37:00+00:00


In [30]:
# Dynamoへ追加する

In [32]:
import json
import boto3

def store_df_q(df_q, dynamodb=None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb', endpoint_url="http://localhost:8000")

    table = dynamodb.Table('Quarter')
    df_q["ticker"] = ticker
    df_q = df_q.drop(["end_date","updated_date","edinet_updated_date","tdnet_updated_date", "to_datetime"], axis=1)
    for index, row in df_q.iterrows():
        # RANGE KEY の設定
        row["fiscal_year_quarter"] = f"{int(row['fiscal_year'])}Q{int(row['fiscal_quarter'])}"
        #print(f"Adding movie : {ticker}{fiscal_year}{fiscal_quarter}")
        table.put_item(Item=row.to_dict())

In [33]:
# if __name__ == '__main__':
store_df_q(df_q)

In [34]:
# 項目を読み込む

In [35]:
from pprint import pprint
import boto3
from botocore.exceptions import ClientError

def get_df_q(ticker:int, fiscal_year_quarter:str, dynamodb=None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb', endpoint_url="http://localhost:8000")

    table = dynamodb.Table('Quarter')

    try:
        response = table.get_item(Key={'ticker': ticker, 'fiscal_year_quarter': fiscal_year_quarter})
    except ClientError as e:
        print(e.response['Error']['Message'])
    else:
        return response['Item']

df_q = get_df_q(ticker, "2015Q4")
if df_q:
    print("Get Quarter succeeded:")
    pprint(df_q)

Get Quarter succeeded:
{'accounting_standard': 'US GAAP',
 'accounts_payable': None,
 'accounts_receivable': None,
 'accounts_receivable_turnover': Decimal('72.87949910463469'),
 'accrual': Decimal('-55800000000'),
 'additional_capital_stock': None,
 'adjusted_debt_ratio': Decimal('77.99573294452318'),
 'advances_received': None,
 'amortization': None,
 'assets': Decimal('1384470000000'),
 'bonds_issuance': None,
 'bonds_payable': None,
 'bonds_repayment': None,
 'bps': Decimal('1288.2979797129763'),
 'buildings': None,
 'capital_stock': Decimal('87784000000'),
 'cash_and_deposits': Decimal('305942000000'),
 'cash_assets_ratio': Decimal('22.098131414909677'),
 'cash_monthly_sales_ratio': Decimal('3.115789831026318'),
 'cash_translation_difference': None,
 'ccc': Decimal('75.54694450604457'),
 'ceo_name': '代表取締役会長兼社長\u3000永守 重信',
 'commercial_papers_liabilities': None,
 'company_name': '日本電産株式会社',
 'construction_in_progress': None,
 'convertible_bond_type_bonds_with_subscription_rights'

In [ ]:
# クエリ - tickerのみ

In [36]:
import boto3
from boto3.dynamodb.conditions import Key # Boto 3 SDK によって Attr が自動的に作成される

def get_df_q(ticker, dynamodb=None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb', endpoint_url="http://localhost:8000")

    table = dynamodb.Table('Quarter')
    response = table.query(
        KeyConditionExpression=Key('ticker').eq(ticker)
    )
    return response['Items']

print(f"Data with ticker : {ticker}")
df_q_list = get_df_q(ticker)
for df_q in df_q_list:
    pprint(df_q)

Data with ticker : 6594
{'accounting_standard': 'US GAAP',
 'accounts_payable': None,
 'accounts_receivable': None,
 'accounts_receivable_turnover': Decimal('72.87949910463469'),
 'accrual': Decimal('-55800000000'),
 'additional_capital_stock': None,
 'adjusted_debt_ratio': Decimal('77.99573294452318'),
 'advances_received': None,
 'amortization': None,
 'assets': Decimal('1384470000000'),
 'bonds_issuance': None,
 'bonds_payable': None,
 'bonds_repayment': None,
 'bps': Decimal('1288.2979797129763'),
 'buildings': None,
 'capital_stock': Decimal('87784000000'),
 'cash_and_deposits': Decimal('305942000000'),
 'cash_assets_ratio': Decimal('22.098131414909677'),
 'cash_monthly_sales_ratio': Decimal('3.115789831026318'),
 'cash_translation_difference': None,
 'ccc': Decimal('75.54694450604457'),
 'ceo_name': '代表取締役会長兼社長\u3000永守 重信',
 'commercial_papers_liabilities': None,
 'company_name': '日本電産株式会社',
 'construction_in_progress': None,
 'convertible_bond_type_bonds_with_subscription_rights

 'headquarters_address': '京都市南区久世殿城町３３８番地',
 'impairment_loss': None,
 'income_before_income_taxes': Decimal('107771000000'),
 'income_before_taxes': None,
 'income_taxes': Decimal('25434000000'),
 'increase_in_properties': Decimal('0'),
 'increase_trade_payables_op_cf': None,
 'intangible_assets': None,
 'interest_and_dividends_income': None,
 'interest_coverage_ratio': None,
 'interest_expense': None,
 'interest_income': None,
 'interest_op_income_ratio': Decimal('0'),
 'inventories': Decimal('184901000000'),
 'inventory_turnover': Decimal('73.94297781892486'),
 'investment_cash_flow': Decimal('-56929000000'),
 'investment_securities': None,
 'investments_and_other_assets': None,
 'issued_share_num': Decimal('298142234'),
 'land': None,
 'lease_and_guarantee_deposits': None,
 'lending': None,
 'liabilities': Decimal('649021000000'),
 'long_term_debt_issuance': None,
 'long_term_debt_repayment': None,
 'long_term_loans_payable': Decimal('116457000000'),
 'loss_of_sales_non_current_ass

 'additional_capital_stock': None,
 'adjusted_debt_ratio': Decimal('89.55392846250297'),
 'advances_received': None,
 'amortization': None,
 'assets': Decimal('1772630000000'),
 'bonds_issuance': None,
 'bonds_payable': None,
 'bonds_repayment': None,
 'bps': Decimal('1549.3232818864483'),
 'buildings': None,
 'capital_stock': Decimal('87784000000'),
 'cash_and_deposits': Decimal('287172000000'),
 'cash_assets_ratio': Decimal('16.200335095310358'),
 'cash_monthly_sales_ratio': Decimal('2.3157631594863215'),
 'cash_translation_difference': None,
 'ccc': Decimal('75.12527993370587'),
 'ceo_name': '代表取締役会長兼社長\u3000永守 重信',
 'commercial_papers_liabilities': None,
 'company_name': '日本電産株式会社',
 'construction_in_progress': None,
 'convertible_bond_type_bonds_with_subscription_rights': None,
 'convertible_bonds': None,
 'corporate_tax_payable': None,
 'cost_of_sales': Decimal('541189000000'),
 'current_allowance_doubtful_accounts': None,
 'current_assets': Decimal('942443000000'),
 'current_dta

 'net_debt_net_income_ratio': Decimal('0.38477001391075716'),
 'net_income': Decimal('131521000000'),
 'net_income_growth_rate_forecast': Decimal('10.827460751792351'),
 'net_income_progress': Decimal('90.23034482758621'),
 'net_profit_margin': Decimal('8.792075748106633'),
 'net_sales': Decimal('1488090000000'),
 'net_sales_growth_rate_forecast': Decimal('5.840372558111406'),
 'net_sales_operating_cash_flow_ratio': Decimal('11.798211129703176'),
 'net_sales_per_employee': Decimal('13835747.624449113'),
 'net_sales_progress': Decimal('94.48190476190476'),
 'net_short_term_debt': Decimal('-178724000000'),
 'non_controling_interests': Decimal('687000000'),
 'non_controlling_interests': Decimal('9890000000'),
 'non_current_allowance_doubtful_accounts': None,
 'non_current_assets': Decimal('856470000000'),
 'non_current_bonds_with_subscription_right': None,
 'non_current_dta': Decimal('11055000000'),
 'non_current_dtl': Decimal('28922000000'),
 'non_current_lease_obligations': None,
 'non_

In [37]:
pd.DataFrame.from_dict(df_q_list)

,treasury_stock,edinet_title,good_will,ex_ordinary_income,ccc,gross_margin,loss_of_sales_non_current_assets,inventory_turnover,work_in_process,net_debt,...,net_income_growth_rate_forecast,notes_accounts_receivable,long_term_loans_payable,net_income,ceo_name,investments_and_other_assets,sga_ratio,r_and_d_ratio,impairment_loss,net_short_term_debt
0,-12111000000,【E01975】日本電産株式会社 有価証券報告書－第43期(平成27年4月1日－平成28年3...,None,None,75.54694450604457,22.91278038513439,None,73.89603340705992,None,-5160000000,...,6.742184947173513,235269000000,136894000000,92863000000,代表取締役会長兼社長 永守 重信,None,12.343395938181603,4.411307912313607,None,None
1,-12115000000,【E01975】日本電産株式会社 四半期報告書－第44期第1四半期(平成28年4月1日－平成...,None,None,66.06105419159017,23.155181277741974,None,63.61181201141649,None,9955000000,...,344.6259244135929,248015000000,128414000000,22264000000,代表取締役会長兼社長 永守 重信,None,11.736167932629995,4.5835354771438706,None,None
2,-12121000000,【E01975】日本電産株式会社 四半期報告書－第44期第2四半期(平成28年7月1日－平成...,None,None,57.636104436173625,24.020530822828572,None,63.65020296587653,None,-30860000000,...,99.56096587507484,253808000000,62510000000,50516000000,代表取締役会長兼社長 永守 重信,None,11.786429799833343,4.5391202595606615,None,None
3,-12134000000,【E01975】日本電産株式会社 四半期報告書－第44期第3四半期(平成28年10月1日－平...,None,None,74.59776351835434,24.099199749374588,None,73.94297781892486,None,-33819000000,...,28.61657561429727,315523000000,116457000000,82337000000,代表取締役会長兼社長 永守 重信,None,11.867735203195474,4.464265146942969,None,None
4,-12143000000,【E01975】日本電産株式会社 訂正有価証券報告書－第44期(平成28年4月1日－平成29...,None,None,90.16417344886256,23.89672070046885,None,84.49649123768098,None,90851000000,...,11.885858522569626,326168000000,161785000000,112671000000,代表取締役会長兼社長 永守 重信,None,12.195752394499843,4.403111453159355,None,None
5,-17293000000,【E01975】日本電産株式会社 四半期報告書－第45期第1四半期(平成29年4月1日－平成...,None,None,70.2368665993657,24.576570064501837,None,66.26440382587886,None,85419000000,...,350.4504504504505,359858000000,282015000000,28298000000,代表取締役会長兼社長 永守 重信,None,13.202911180998628,3.860200355007855,None,None
6,-17304000000,【E01975】日本電産株式会社 四半期報告書－第45期第2四半期(平成29年7月1日－平成...,None,None,75.12527993370587,24.403330120549246,None,72.881489840989,None,98218000000,...,113.0705463261977,393612000000,335261000000,60384000000,代表取締役会長兼社長 永守 重信,None,12.863568425316737,3.7420553436980546,None,None
7,-17320000000,【E01975】日本電産株式会社 四半期報告書－第45期第3四半期(平成29年10月1日－平...,None,None,73.73675418005169,24.25851394448609,None,77.06902082705511,None,86219000000,...,35.07381572976796,396432000000,335937000000,95300000000,代表取締役会長兼社長 永守 重信,None,12.76338906666932,3.7078597838362772,None,None
8,-19151000000,【E01975】日本電産株式会社 有価証券報告書－第45期(平成29年4月1日－平成30年3...,359328000000,None,66.56540399920688,23.959101936038813,None,73.47758884439759,None,50341000000,...,10.827460751792351,388741000000,314631000000,131521000000,代表取締役会長（最高経営責任者） 永守 重信,None,9.021833356853417,3.725446713572432,None,-178724000000
9,-22056000000,【E01975】日本電産株式会社 四半期報告書－第46期第1四半期(平成30年4月1日－平成...,None,None,74.91516173713437,25.15427579703565,None,78.50343601001673,None,92277000000,...,294.2392791053182,388141000000,316908000000,37487000000,代表取締役会長（最高経営責任者) 永守 重信,None,8.81004129245103,4.074439247440378,None,33515000000


In [11]:
# TODO : Quater の start end を算出する
#  oldest_fiscal_year / oldest_fiscal_quarter= 2004.0 1.0
# latest_fiscal_year / latest_fiscal_quarter = 2020.0 2.0
# oldest から latest まで、3年分のレンジを自動算出する ex. generate_df_q("7974", "2015Q4", "2018Q3")